In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import json
import matplotlib.pyplot as plt
import numpy as np


In [0]:
### data pre-processing -- read data from files - single 
# json analyis 
participant= 1
fileAdd = './cleanjson/P'+str(participant)+'.json'
data = pd.read_json(fileAdd)
# print(data['data'])
test = json_normalize(data['data'])
# print(test)
df = pd.DataFrame.from_dict(test, orient='columns')

L1 = df.loc[(df['index']<=21)|(df['index']>=24)&(df['index']<=45)]
L2 = df.loc[(df['index']>=46)&(df['index']<=99)]
L3 = df.loc[(df['index']>=120)&(df['index']<=153)]
print(L1.describe())
print(L2.describe())
print(L3.describe())

In [0]:
### data pre-processing -- read data from file - group
M1 = []
M2 = []
M3 = []
for p in range(1,15):
    fileAdd = './cleanjson/P'+str(p)+'.json'
    data = pd.read_json(fileAdd)
    test = json_normalize(data['data'])
    df = pd.DataFrame.from_dict(test, orient='columns')
    L1 = df.loc[(df['index']<=21)|(df['index']>=24)&(df['index']<=45)]
    L2 = df.loc[(df['index']>=46)&(df['index']<=99)]
    L3 = df.loc[(df['index']>=120)&(df['index']<=153)]  

    mean_L1 = L1['score'].mean()
    mean_L2 = L2['score'].mean()
    mean_L3 = L3['score'].mean()
    M1.append(mean_L1)
    M2.append(mean_L2)
    M3.append(mean_L3)
print(M1)
print(M2)
print(M3)

In [2]:
### data pre-processing -- data visualization
Mean_data = {'L1':M1,
'L2':M2,
'L3':M3}
df_accuracy = pd.DataFrame(Mean_data)
print(df_accuracy.mean())
fig =plt.figure()
df_accuracy.plot()
fig =plt.figure()
df_accuracy.boxplot(grid=False)

NameError: name 'M1' is not defined

In [0]:
### one way anova  by stats 
# load packages
import scipy.stats as stats
d = df_accuracy
# stats f_oneway functions takes the groups as input and returns F and P-value
fvalue, pvalue = stats.f_oneway(d['L1'], d['L2'], d['L3'])
print(fvalue, pvalue)


# get ANOVA table as R like output
import statsmodels.api as sm
from statsmodels.formula.api import ols
# reshape the d dataframe suitable for statsmodels package 
d_melt = pd.melt(d.reset_index(), id_vars=['index'], value_vars=['L1', 'L2', 'L3'])
# replace column names
d_melt.columns = ['index', 'Levels', 'value']
# Ordinary Least Squares (OLS) model
model = ols('value ~ C(Levels)', data=d_melt).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

In [0]:
### one way anova and post hoc test  by pingouin 
import pingouin as pg
from pingouin import pairwise_tukey
# perform multiple pairwise comparison (Tukey HSD)
# for unbalanced (unequal sample size) data, pairwise_tukey uses Tukey-Kramer test
anova = d_melt.anova(dv='value', between='Levels', detailed=False,effsize='n2')
print(anova)
print()
m_comp = pairwise_tukey(data=d_melt, dv='value', between='Levels')
print(m_comp)

In [0]:
### within subjects data arrangement 
subject = range(1,15)
L1_fill = np.full((14),1)
data1 = {'Subjects':subject,'Levels':L1_fill,'Accuracy':M1}
df_r = pd.DataFrame(data1)
# print(df_r)
L1_fill = np.full((14),2)
data2 = {'Subjects':subject,'Levels':L1_fill,'Accuracy':M2}
df_r2 = pd.DataFrame(data2)
df_all = df_r.append(df_r2,ignore_index=True)
# print(df_r2)
L1_fill = np.full((14),3)
data3 = {'Subjects':subject,'Levels':L1_fill,'Accuracy':M3}
df_r3 = pd.DataFrame(data3)
df_all  = df_all.append(df_r3,ignore_index=True)
print(df_all)

In [0]:
### within subjects data visualization 
import seaborn as sns
sns.set()
sns.pointplot(data=df_all, x='Levels', y='Accuracy',dodge=True, capsize=.1, errwidth=1 )

In [0]:
### repeated measure anova by pingouin

import pingouin as pg
# Compute the two-way mixed-design ANOVA
aov = pg.rm_anova(dv='Accuracy', within='Levels', subject='Subjects', data=df_all,effsize='n2')

# Pretty printing of ANOVA summary
pg.print_table(aov)


In [14]:
from pingouin import friedman, read_dataset

friedman(dv='Accuracy', within='Levels', subject='Subjects', data=df_all)


,Source,ddof1,Q,p-unc
Friedman,Levels,2,2.714286,0.257395


In [0]:
## old methods 

# from statsmodels.stats.anova import AnovaRM

# #perform the repeated measures ANOVA
# print(AnovaRM(data=df_all, depvar='Accuracy', subject='Subjects', within=['Levels']).fit())

In [ ]:
import pingouin as pg
L12 = allFeaturers.loc[(allFeaturers['Levels']==1)|(allFeaturers['Levels']==2)]
L13 = allFeaturers.loc[(allFeaturers['Levels']==1)|(allFeaturers['Levels']==3)]
L23 = allFeaturers.loc[(allFeaturers['Levels']==2)|(allFeaturers['Levels']==3)]

pg.wilcoxon(L12['su'],L12[''], tail='two-sided')